In [5]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from io import BytesIO

np.random.seed(0)

In [6]:
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")

In [12]:
field1 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox")
field2 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature")
field3 = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("label_feature")

In [49]:
def make_offset(vector):
    return [vector[i] - vector[i - 1] for i in range(1, len(vector))]

In [61]:
def group_objects(objects, cardinality_list):
    result = []
    index = 0
    for count in cardinality_list:
        if count == 0:
            result.append(None)
        if count > 0:  # Only create a group if count > 0
            result.append(objects[index:index+count])
            index += count
        if index > len(objects):
            result.append(None)
    return result

In [82]:
cardinality_list = make_offset(table.column("image_feature").chunk(0).values.field("boundingbox_feature").offsets.to_pylist())
objects = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.to_pylist()

new_struct = pa.array(group_objects(objects, cardinality_list)).values

In [103]:
import pyarrow as pa


# Sample deeply nested pyarrow Table with a nested struct field
nested_data = pa.table({
    'id': [1, 2, 3],
    'info': pa.array([
        {'age': {'birth_year': 1995}, 'score': 90},
        {'age': {'birth_year': 1990}, 'score': 85},
        {'age': {'birth_year': 2000}, 'score': 88}
    ], type=pa.struct([
        ('age', pa.struct([('birth_year', pa.int32())])),
        ('score', pa.int32())
    ]))
})


# Step 1: Extract the inner 'birth_year' field from the nested 'age' struct
#info_column = nested_data['info']  # This is a pyarrow.StructArray
#age_struct = info_column.chunk(0).field('age')  # Extract the 'age' struct
#birth_year_column = age_struct.field('birth_year')  # Extract 'birth_year' field


# Step 2: Perform operations (e.g., calculate age based on current year)
#current_year = 2024
#birth_years = birth_year_column.to_pandas()  # Convert to pandas for easy manipulation
#calculated_ages = current_year - birth_years  # Calculate age


# Step 3: Create a new pyarrow.Array for 'calculated_ages'
#calculated_ages_column = pa.array(calculated_ages)


# Step 4: Recreate the 'age' struct with both 'birth_year' and 'calculated_age'
#new_age_struct = pa.StructArray.from_arrays(
#    [birth_year_column, calculated_ages_column],  # Both fields
#    names=['birth_year', 'calculated_age']  # New child field added
#)

new_field = pa.array([73 for i in range(50)])

new_struct_no_cardinality = pa.StructArray.from_arrays(
    [field1,field2, field3, new_field ],  # Both fields
    names=['bbox', 'image_1_feature',"label_feature","area"]  # New child field added
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).values.field("boundingbox_feature").offsets.to_pylist())
new_struct = pa.array(group_objects(new_struct_no_cardinality.to_pylist(), cardinality_list))

# Step 5: Replace the 'age' struct in the 'info' struct
field4 = table.column("image_feature").chunk(0).values.field("image")
field5 = table.column("image_feature").chunk(0).values.field("shape")
#field6 = table.column("image_feature").values.field("boundingbox_feature")
field6 = table.column("image_feature").chunk(0).values.field("text_1_feature")

new_image_feature_column_no_cardinality = pa.StructArray.from_arrays(
    [new_struct,field4,field5,field6],  # Attach the updated 'age' struct
    names=["boundingbox_feature", "image","shape","text_1_feature"]
)

cardinality_list = make_offset(table.column("image_feature").chunk(0).offsets.to_pylist())
new_image_feature_column = pa.array(group_objects(new_image_feature_column_no_cardinality.to_pylist(), cardinality_list))



# Step 6: Replace the original 'info' column in the table with the updated one
new_table = table.set_column(0, 'image_feature', new_image_feature_column)


# Print the updated table
print(new_table)

pyarrow.Table
image_feature: list<item: struct<boundingbox_feature: list<item: struct<area: int64, bbox: list<item: double>, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>>
  child 0, item: struct<boundingbox_feature: list<item: struct<area: int64, bbox: list<item: double>, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>
      child 0, boundingbox_feature: list<item: struct<area: int64, bbox: list<item: double>, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int64>>>>
          child 0, item: struct<area: int64, bbox: list<item: double>, image_1_feature: list<item: struct<image